In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

"""
Prepare consolidated data

"""

# Import libraries

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pandas as pd
import aerospike
import json

print(__doc__)

class Consolidate:
    
    def __init__(self):
        pass
        
    def read(self, file):
        data = pd.read_excel(file)
        return data

    def clean_data(self, data):
    
        # Select numeric data alone from data object
        numeric_data = data.loc[:, data.dtypes != object]
    
        # Select string data alone from data object to trim
        trim_data = data.select_dtypes(['object'])
    
        # Trim all string objects
        trim_data = trim_data.apply(lambda x: x.str.strip())
    
        # Merge numeric and string data in to data object
        data = pd.concat([numeric_data, trim_data], axis=1)
    
        # Change column name to lower case
        data.columns = [x.lower() for x in data.columns]
        return data
    
    def load_data(self, data):
            
        # Split dataframe into key and records
        key = data['vehicle_no']
    
        record_json = data.to_dict(orient='records')
    
        config = {
            'hosts': [ ('127.0.0.1', 3000) ]
        }

        # Create a client and connect it to the cluster
        try:
            client = aerospike.client(config).connect()
        except:
            import sys
            print("failed to connect to the cluster with", config['hosts'])
            sys.exit(1)
    
        # Records are addressable via a tuple of (namespace, set, key)
        for pk, bin in zip(key, record_json):
            string = "('dev', 'customer_data', "
            string +=`pk`
            string = string + ")"
        
            from ast import literal_eval as make_tuple
            string = make_tuple(string)
        
            try:
                client = aerospike.client(config).connect()
                # Write a record
                client.put(string, bin)
            except Exception as e:
                import sys
                print("error: {0}".format(e), file=sys.stderr)

            # Read a record after insert for verification
            (key, metadata, record) = client.get(string)
            print (key, record)
        
            # Close the connection to the Aerospike cluster
            client.close()
    
    def remove_data(self, data):

        key = data['vehicle_no']
    
        config = {
          'hosts': [ ('127.0.0.1', 3000) ]
        }

        # Create a client and connect it to the cluster
        try:
            client = aerospike.client(config).connect()
        except:
            import sys
            print("failed to connect to the cluster with", config['hosts'])
            sys.exit(1)
    
        # Records are addressable via a tuple of (namespace, set, key)
        for pk in key:
            string = "('dev', 'customer_data', "
            string +=`pk`
            string = string + ")"
        
            from ast import literal_eval as make_tuple
            string = make_tuple(string)
        
            try:
                client = aerospike.client(config).connect()
                # Remove a record
                client.remove(string)
            except Exception as e:
                import sys
                print("error: {0}".format(e), file=sys.stderr)

            # Close the connection to the Aerospike cluster
            client.close()
    
def drive(mode):
    
    cons = Consolidate()

    # Read customer details excel
    data1 = cons.read("/home/azureuser/Aerospike/data/customer/customer details.xls")
    
    # Clean data1
    data1 = cons.clean_data(data1)
    #print (data1.columns)
    
    # Read Customer vehicle excel
    data2 = cons.read("/home/azureuser/Aerospike/data/customer/Customer_vehicle.xls")
    
    # Clean data2
    data2 = cons.clean_data(data2)
    #print (data2.columns)
    
    # Merge data1 and data2 by cust_id
    data2 = pd.merge(data1, data2, on='cust_id', how='inner')
    
    # Read Motor vehicle report excel
    data3 = cons.read("/home/azureuser/Aerospike/data/customer/Motor_vehicle_report.xls")
    data3 = data3.drop('Gender',1)
    # Clean data3
    data3 = cons.clean_data(data3)
    
    # Merge data2 and data3 by vehicle_no
    data3 = pd.merge(data2, data3, on='vehicle_no', how='inner')
    #print (data3.columns)
    
    # Read Customer address excel
    data4 = cons.read("/home/azureuser/Aerospike/data/customer/customer_address.xls")
    
    # Clean data4
    data4 = cons.clean_data(data4)
    
    # Merge data3 and data4 by cust_id
    data4 = pd.merge(data3, data4, on='cust_id', how='inner')
    #print (data4.columns)
    data4 = data4.drop('dob',1)
    #print (data4.columns)
    #print (data4.count)
    
    if mode == 'load':
        # Load data in to aerospike
        cons.load_data(data4)
    else:
        # Remove data from aerospike
        cons.remove_data(data4)
    
if __name__ == '__main__':
    drive('load')


Prepare consolidated data


('dev', 'customer_data', None, bytearray(b'\xa5&\x82\x10q\x8d\xb1\xac\xd4\xed\x08y\xdc\x1d\xb0\xcb\xef\x06?\x9d')) {'child2_d_o_b': nan, 'spouse_name': nan, 'last_name': 'LOWE', 'weight': 61, 'email_id': 'Quatasia@yahoo.com', 'vin': 'SHSRD78494U256469', 'pincode': 52531, 'height': 6.8, 'child1_name': 'Lillian', 'no_of_violatn': 0, 'no_of_child': 1, 'city': 'Chicago', 'first_name': 'Quatasia', 'credit_rating': 571, 'marital_sts': 'Separated', 'child1_d_o_b': Timestamp('1950-02-01 00:00:00'), 'state': 'Illinois', 'date_of_birth': Timestamp('1982-10-21 00:00:00'), 'ssn_no': '65-65-3156', 'model_id': 'M00039', 'license_susp': 'No', 'job_type': 'Professional', 'spouse_d_o_b': Timestamp('1984-10-21 00:00:00'), 'cust_id': 2396305636, 'no_of_accidnt': 0, 'dt_of_purchse': Timestamp('2007-09-09 00:00:00'), 'designation': 'CTO', 'class_id': 'CCLA', 'gender': 'F', 'age': 34, 'vehicle_no': 76990, 'child2_name': nan, 'eye_and_hair': 'Black', 'payment_type': 'Buy', 'count

### Create index for table dev.customer_data

Syntax - CREATE INDEX numindex ON test.testset (binB) NUMERIC|STRING

###### Index 'cust_id'
create index i_cust_id on dev.customer_data (cust_id) numeric

###### Index 'vehicle_no'
create index i_vehicle_no on dev.customer_data (vehicle_no) numeric